# OpenAI Multi-Agents SDK Demo

OpenAI have released an [Agents SDK](https://openai.github.io/openai-agents-python/) and [Support MCP](https://openai.github.io/openai-agents-python/mcp/). This demo want to show how to use the SDK to make an application that call multiple agents to do a task.

Install the dependencies library:

In [2]:
%pip install -qU openai-agents python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Steup LLM and Agent

Setup OpenAI API url, model and key:

In [ ]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from .env file
load_dotenv()

# Get API base URL and model with default values
OPENAI_API_BASE = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

# Set up OpenAI API configuration
# Try to get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If not found, ask user to input it
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

OPENAI_MODEL

'qwen2.5:7b'

In [2]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI, RunResultStreaming
from agents import function_tool
from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
    ResponseTextDeltaEvent,  # text streaming
)

async def show_streaming_response(
    response: RunResultStreaming,
) -> None:
    async for event in response.stream_events():
        if event.type == "raw_response_event": 
            if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
                # this is streamed parameters for our tool call
                print(event.data.delta, end="", flush=True)
            elif isinstance(event.data, ResponseTextDeltaEvent):
                # this is streamed final answer tokens
                print(event.data.delta, end="", flush=True)
        elif event.type == "agent_updated_stream_event":
            # this tells us which agent is currently in use
            print(f"> Current Agent: {event.new_agent.name}")
        elif event.type == "run_item_stream_event":
            # these are events containing info that we'd typically
            # stream out to a user or some downstream process
            if event.name == "tool_called":
                # this is the collection of our _full_ tool call after our tool
                # tokens have all been streamed
                print()
                print(f"> Tool Called, name: {event.item.raw_item.name}")
                print(f"> Tool Called, args: {event.item.raw_item.arguments}")
            elif event.name == "tool_output":
                # this is the response from our tool execution
                print(f"> Tool Output: {event.item.raw_item['output']}")

model= OpenAIChatCompletionsModel(
    model=OPENAI_MODEL,
    openai_client=AsyncOpenAI(base_url=OPENAI_API_BASE,api_key=OPENAI_API_KEY),
)

@function_tool
def add_tool(x: float, y: float) -> float:
    """Add X and Y to get the exact result.
    """
    return x + y

@function_tool
def multiply_tool(x: float, y: float) -> float:
    """Multiplies `x` and `y` to provide a precise
    answer."""
    return x*y

math_agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model=model,
    tools=[add_tool,multiply_tool]
)

[non-fatal] Tracing: server error 500, retrying.


## Call Multiple Tools in a Conversation

Our step-by-step approach will allow the LLM itself to recognize which tool should be used to process the task, so we'll start by having the Agent compute an addition and then a multiplication:

In [3]:
response = Runner.run_streamed(
    starting_agent=math_agent,
    input="What is 1 + 2?",
)

await show_streaming_response(response)

> Current Agent: Assistant
{"x":1,"y":2}
> Tool Called, name: add_tool
> Tool Called, args: {"x":1,"y":2}
> Tool Output: 3.0
The result of 1 + 2 is 3.0.

In [4]:
response.to_input_list()
response = Runner.run_streamed(
    starting_agent=math_agent,
    input= response.to_input_list()+[
        {"role": "user", "content": "What is last asswer * 5?"}
    ]
)
await show_streaming_response(response)

> Current Agent: Assistant
{"x":3,"y":5}
> Tool Called, name: multiply_tool
> Tool Called, args: {"x":3,"y":5}
> Tool Output: 15.0
The result of 3 * 5 is 15.0.

## Call Multiple Tools in a request

In [5]:
response = Runner.run_streamed(
    starting_agent=math_agent,
    input="What is the result of calculating 1+2 and then multiplying the result by 5?",
)
await show_streaming_response(response)

> Current Agent: Assistant
{"x":1,"y":2}
> Tool Called, name: add_tool
> Tool Called, args: {"x":1,"y":2}
> Tool Output: 3.0
{"x":3,"y":5}
> Tool Called, name: multiply_tool
> Tool Called, args: {"x":3,"y":5}
> Tool Output: 15.0
The result of calculating \( 1 + 2 \) and then multiplying the result by \( 5 \) is \( 15 \).

But we will find that when our arithmetic becomes complex, LLM's processing often breaks down:

In [29]:
response = Runner.run_streamed(
    starting_agent=math_agent,
    input="What is 3+2*2+5?",
)
await show_streaming_response(response)

> Current Agent: Assistant
{"x":3,"y":4}
> Tool Called, name: add_tool
> Tool Called, args: {"x":3,"y":4}
> Tool Output: 7.0
The result of the expression 3 + 2 * 2 + 5 is 10. Let me break it down for you: First, according to the order of operations (PEMDAS/BODMAS), we perform the multiplication operation 2*2 which equals 4, then add 3 and 5 to get the final result of 10.

##  Define other agent

In [ ]:
breakdown_agent = Agent(
    name="Assistant",
    instructions="Please show step-by-step how to solve the following math problem and explain your calculations at each step. Please do not calculate the final answer. Here is an example: Equation: 3+2×3+5 Step 1: First identify and perform the multiplication operation. 2 x 3 = ? Step 2: Substitute the result of the previous step into the original equation. After substituting the result, the equation becomes: 3+? +5 (fill in the specific values here) Step 3: Perform the addition operations in order from left to right. First, calculate 3+? =? Then calculate (? +5)=? Please show your calculations step-by-step following the above steps and allow the user to complete the final calculations themselves.",
    model=model,
)

response = Runner.run_streamed(
    starting_agent=breakdown_agent,
    input="3+2*3+5",
)

await show_streaming_response(response)

> Current Agent: Assistant
Step 1: Identify and perform the multiplication operation first.
- The multiplication in the equation is \(2 \times 3\).
- Calculate \(2 \times 3 = ?\)

Step 2: Substitute the result of the previous step into the original equation.
- After calculating, we find that \(2 \times 3 = 6\).
- Now substitute \(6\) for \(2 \times 3\) in the original equation. The equation becomes:
   - \(3 + 6 + 5\)

Step 3: Perform the addition operations from left to right.
- First, calculate \(3 + 6 = ?\)
- Then, take the result and add \(5\) to it.

Let's continue this step-by-step, filling in the values as we go:
1. Calculate \(3 + 6 = ?\)
2. Take that result and calculate \((? + 5) = ?\)

In [30]:
response = Runner.run_streamed(
    starting_agent=breakdown_agent,
    input="3+2*3+5",
)

await show_streaming_response(response)

response = Runner.run_streamed(
    starting_agent=math_agent,
    input=response.to_input_list()+[
        {"role": "user", "content": "What is the final answer?"}
    ]
)

await show_streaming_response(response)

> Current Agent: Assistant
Sure! Let's solve this equation step by step.

### Step 1: Identify and perform the multiplication operation.
The multiplication in the equation is \(2 \times 3\).
\[2 \times 3 = ?\]
Calculate this to get:
\[2 \times 3 = 6\]

### Step 2: Substitute the result of the previous step into the original equation.
After substituting, the equation becomes:
\[3 + 6 + 5\]

Now you can proceed with the addition operations. Please go ahead and perform these steps in order from left to right.

You need to first calculate \(3 + 6 = ?\) and then use that result to find \(? + 5\).> Current Agent: Assistant
{"x":3,"y":6}{"x":9,"y":5}
> Tool Called, name: add_tool
> Tool Called, args: {"x":3,"y":6}

> Tool Called, name: add_tool
> Tool Called, args: {"x":9,"y":5}
> Tool Output: 9.0
> Tool Output: 14.0
The final answer to the equation \(3 + 2 \times 3 + 5\) is \(14\). 

Here’s a breakdown:
1. First, we calculated \(2 \times 3 = 6\).
2. Then added \(3 + 6 = 9\).
3. Finally, addi